In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pysolr
from IPython.core.display import display, HTML

import katdal
import katpoint

import os
import glob

import csv

archive = pysolr.Solr('http://192.168.1.50:8983/solr/kat_core')

ptuse_string = 'beamform_single_pulsar.py'
flux_string = 'beamform_flux.py'

In [3]:
Puls='J1909-3744'
Puls='J1939+2134'
Puls='J0437-4715'
#Puls='All'
delayfilenm=Puls+".delays.csv"
outputfilenm=Puls+".Log.csv"
outputfile=open(outputfilenm,"w")
delayfile = open(delayfilenm,'w')


In [4]:
def get_delays(filename):
    
    data = katdal.open(filename)
    ref=data.ref_ant
    utc=data.start_time
    antnames=[]
    for ant in data.ants:
        antnames.append(ant.name)
        if ant.name==data.ref_ant:
            HDel=ant.delay_model.params['FIX_H'].value
            VDel=ant.delay_model.params['FIX_V'].value
    ants=" ".join(antnames)

    NAnts=len(antnames)
    array_ant = katpoint.Antenna('ref', *(data.ants[0].ref_position_wgs84))
    #print array_ant,NAnts
    correlator_model = katpoint.DelayCorrection(data.ants, array_ant)
    delay=correlator_model.max_delay
    
    return delay,utc,NAnts,ants,ref,HDel,VDel

In [5]:
def get_obsreport_link(scheduleID):

    ar_res = archive.search('CAS.ProductTypeName:MeerKATAR1ReductionProduct AND ExperimentID:%s' % scheduleID, sort="StartTime desc")    
    if ar_res.hits > 0:
        redfiles = ar_res.docs[0]["CAS.ReferenceDatastore"] 

        for fl in redfiles:
            if fl.endswith('html') or fl.endswith('pdf'):
               
                link=fl[5:]
                obsrep=link.replace('/var/kat','http://kat-archive.kat.ac.za')
    else:

        obsrep="None"
    
    return obsrep

In [6]:
def get_reduction_metadata(filename, reduction_name=None):
    #Get all reduction products from filename  with given reduction_name 
    #(or all reduction products if reduction_name is None)
    mysolr = pysolr.Solr('http://kat-archive.kat.ac.za:8983/solr/kat_core')
    fn_search_result = mysolr.search('Filename:'+filename)
    if fn_search_result.hits < 1:
        return []
    CASProductId = fn_search_result.docs[0]['CAS.ProductId']
    reduction_products = mysolr.search('InputDataProductId:'+CASProductId)
    if reduction_name==None:
        return reduction_products.docs
    else:
        return [product for product in reduction_products.docs if product.get('ReductionName')==reduction_name]

In [7]:
# read in previous manual phaseup QA

file='PhaseUp.csv'
result={}
with open(file,'r') as f:
    line=csv.DictReader(f)
    for d in line:
        result.setdefault(d['ScheduleID'],(d['PhaseUp']))


In [8]:
search_string=[]
instr_str = []

instr_str.append('InstructionSet:"%s"'%ptuse_string)
instr_str.append('InstructionSet:"%s"'%flux_string)
instr_string = '(' +' OR '.join(instr_str) + ')'
search_str = []
search_str.append(instr_string)

search_str.append('CAS.ProductTypeName:MeerKATAR1TelescopeProduct')
search_str.append('Targets:"%s"'%Puls)

search_str.append('StartTime:[2016-08-01T00:00:00Z TO 2018-10-01T00:00:00Z]')
search_string = ' AND '.join(search_str)

print "Searching archive for %s" %search_str
ar_res = archive.search( search_string ,rows=10000,**{'sort':'StartTime asc'})
print "Found %d hits" %ar_res.hits



Searching archive for ['(InstructionSet:"beamform_single_pulsar.py" OR InstructionSet:"beamform_flux.py")', 'CAS.ProductTypeName:MeerKATAR1TelescopeProduct', 'Targets:"J0437-4715"', 'StartTime:[2016-08-01T00:00:00Z TO 2018-10-01T00:00:00Z]']
Found 461 hits


In [10]:
#f = open('J1909.txt','w')
#g = open('delays.txt','w')
print "Writing observation log to %s" %(outputfilenm)
print "Writing delays to %s" %(delayfilenm)

outputfile=open(outputfilenm,"w")
delayfile = open(delayfilenm,'w')
print "\n\n"

delayfile.write("name;SB;target;NAnts;ants;delay;ref;HDel;VDel\n")

for r in ar_res.docs:
    
    
    SB = r['ScheduleBlockIdCode']
    print "Processing %s" %(SB)
    try:
        Observer = r['Observer']
        target = r['Targets'][0]
    except:
        Observer="None"
        target="None"
        continue
    duration=r['Duration']
    Start=r['StartTime']
    
    Ants=r['Antennas']
    NAnts=len(r['Antennas'])
    VisFile=r['CAS.ReferenceDatastore'][0][5:]
    progress='http://obs.mkat.karoo.kat.ac.za:8081/tailtask/%s/progress'%(SB)
    archivelink = 'http://kat-archive.kat.ac.za:8080/archive_search/#{"ei":"%s","so":true,"a":true,"b":true,"ob":false,"rs":{}}'%(SB)
    instructions = r['InstructionSet']
    QA = 3
    phasesb='None'
    phaseobsrep='None'
    phasesb='None'
    phasesrc='None'
    
    phaseres = archive.search('CAS.ProductTypeName:MeerKATAR1TelescopeProduct AND InstructionSet:bf_phaseup* AND StartTime:[* TO %s]' % (r['StartTime']), sort="StartTime desc")

    if phaseres.hits > 0:
            try:
                QA = phaseres.docs[0]['QA1']
            except:
                QA = 3
            
            phasesb = phaseres.docs[0]['ScheduleBlockIdCode']
            try:
                phasesrc=  phaseres.docs[0]['Targets'][0]
                pharchivelink = 'http://kat-archive.kat.ac.za:8080/archive_search/#{"ei":"%s","so":true,"a":true,"b":true,"ob":false,"rs":{}}'%(phasesb)

                phaseprogress='http://obs.mkat.karoo.kat.ac.za:8081/tailtask/%s/progress'%(phasesb)
            
                PhaseUpTime=phaseres.docs[0]['StartTime']
                
                phaseobsrep=get_obsreport_link(phasesb)
            except:
                phaseobsrep="None"
                phasesrc = "None"

    try:
        (delay,utc,NAnts,ants,ref,HDel,VDel) = get_delays(VisFile)
    except:
        print "Problem with get_delays(%s)" %(VisFile)

    
    search_str = []
    search_str.append('ScheduleBlockIdCode:"%s"'%SB)

    search_str.append('CAS.ProductTypeName:PTUSETimingArchiveProduct')

    search_string = ' AND '.join(search_str)
    
    sb_res = archive.search( search_string ,rows=10000)
    
    name=str(utc)
    ptuseData="None"
    print "PTUSE search %d" %(sb_res.hits)
    if sb_res.hits > 0:
        
        for s in sb_res.docs:
            if s['CAS.ProductTypeName']=='PTUSETimingArchiveProduct':
                duration=s['Duration']
                
                ptuseData=s['CAS.ReferenceDatastore'][0][5:]
                data_files = os.listdir(ptuseData)
                sort=sorted(data_files)
                obs=sort[0].split('.')[0]
                
                obs1=obs.replace('-','',2)

                utc=obs1.replace('-','T').replace(':','',2).split(':')[0]
                name=target+'.'+utc
                
            

                ptuseFileSize=sum(s['CAS.ReferenceFileSize'])/1024.0**3
    
    print "\t\t\t %s" %name
    
    if phasesb in result.keys():
        PU=result[phasesb]
    else:        
        PU=" "

    if (QA < 0):
        print "\t\t\t Phaseup failed"
    if (ptuseData=="None"):
        print "\t\t\t No ptuse data"
    outputfile.write("%s;%s;%s;%s;%s;" %(SB,name,target,Observer,Start))
                                      
    outputfile.write("%d;%d;%s;%s;" %(duration,NAnts,ants,progress))
    outputfile.write("%s;%s;%s;%s;" %(archivelink,VisFile,ptuseData,phasesb))
    outputfile.write("%s;%s;%s;%s;" %(phasesrc,PhaseUpTime,PU,QA))
    outputfile.write("%s;%s;%s;%s\n" %(phaseobsrep,pharchivelink,phaseprogress,instructions))
    

    delayfile.write("%s;%s;%s;%d;%s;%6.3f;%s;%6.3f;%6.3f\n" %(name,SB,target,NAnts,ants,delay*1E6,ref,HDel,VDel))
                
    
        

delayfile.close()   
outputfile.close()  
        

Writing observation log to J0437-4715.Log.csv
Writing delays to J0437-4715.delays.csv



Processing 20160928-0051
PTUSE search 0
			 2016-09-28 21:15:44.154
			 No ptuse data
Processing 20160928-0052
PTUSE search 0
			 2016-09-28 21:30:34.277
			 No ptuse data
Processing 20160928-0053
PTUSE search 0
			 2016-09-28 21:45:34.896
			 No ptuse data
Processing 20160928-0059
PTUSE search 0
			 2016-09-28 23:15:47.606
			 No ptuse data
Processing 20160928-0061
PTUSE search 0
			 2016-09-29 03:15:52.512
			 No ptuse data
Processing 20160928-0062
PTUSE search 0
			 2016-09-29 03:30:37.638
			 No ptuse data
Processing 20160928-0063
PTUSE search 0
			 2016-09-29 03:45:32.259
			 No ptuse data
Processing 20160928-0065
PTUSE search 0
			 2016-09-29 04:15:45.492
			 No ptuse data
Processing 20160928-0066
PTUSE search 0
			 2016-09-29 04:30:34.616
			 No ptuse data
Processing 20160928-0067
PTUSE search 0
			 2016-09-29 04:45:37.234
			 No ptuse data
Processing 20160928-0069
PTUSE search 0
			 2016-09

PTUSE search 0
			 2016-10-13 00:11:40.269
			 No ptuse data
Processing 20161012-0085
PTUSE search 0
			 2016-10-13 01:21:26.748
			 No ptuse data
Processing 20161012-0087
PTUSE search 0
			 2016-10-13 02:31:26.472
			 No ptuse data
Processing 20161012-0090
PTUSE search 0
			 2016-10-13 03:41:12.201
			 No ptuse data
Processing 20161012-0089
PTUSE search 0
			 2016-10-13 03:52:52.904
			 No ptuse data
Processing 20161012-0091
PTUSE search 0
			 2016-10-13 04:04:33.108
			 No ptuse data
Processing 20161012-0093
PTUSE search 0
			 2016-10-13 04:24:14.108
			 No ptuse data
Processing 20161012-0095
PTUSE search 0
			 2016-10-13 05:33:42.345
			 No ptuse data
Processing 20161012-0096


PTUSE search 0
			 2016-10-13 05:45:26.047
			 No ptuse data
Processing 20161020-0021
PTUSE search 0
			 2016-10-20 22:15:58.790
			 No ptuse data
Processing 20161023-0004


PTUSE search 0
			 2016-10-24 00:15:56.831
			 No ptuse data
Processing 20161023-0010
PTUSE search 0
			 2016-10-24 01:53:20.359
			 No ptuse data
Processing 20161027-0011


PTUSE search 0
			 2016-10-27 22:15:43.027
			 No ptuse data
Processing 20161027-0018
PTUSE search 0
			 2016-10-27 23:53:10.554
			 No ptuse data
Processing 20161030-0019


...

PTUSE search 1
			 J0437-4715.20170830T225100
Processing 20170830-0030
PTUSE search 1
			 J0437-4715.20170831T000100
Processing 20170830-0032
PTUSE search 1
			 J0437-4715.20170831T011050
Processing 20170830-0034
PTUSE search 1
			 J0437-4715.20170831T022050
Processing 20170830-0036


PTUSE search 1
			 J0437-4715.20170831T033100
Processing 20170830-0038
PTUSE search 1
			 J0437-4715.20170831T044050
Processing 20170830-0040
PTUSE search 1
			 J0437-4715.20170831T055100
Processing 20170831-0003
PTUSE search 1
			 J0437-4715.20170831T075450
Processing 20170831-0009
PTUSE search 1
			 J0437-4715.20170831T084420
Processing 20170901-0033
PTUSE search 1
			 J0437-4715.20170901T222100
Processing 20170901-0034
PTUSE search 0
			 2017-09-01 22:45:53.917
			 No ptuse data
Processing 20170901-0035
PTUSE search 1
			 J0437-4715.20170901T231050
Processing 20170901-0037
PTUSE search 1
			 J0437-4715.20170901T232610
Processing 20170901-0038
PTUSE search 1
			 J0437-4715.20170901T234720
Processing 20170901-0039
PTUSE search 1
			 J0437-4715.20170902T000820
Processing 20170901-0041
PTUSE search 1
			 J0437-4715.20170902T002350
Processing 20170901-0042
PTUSE search 1
			 J0437-4715.20170902T004450
Processing 20170901-0043
PTUSE search 1
			 J0437-4715.20170902T010600
Processing 20170

PTUSE search 0
			 2017-09-10 02:20:44.156
			 No ptuse data
Processing 20170909-0030
PTUSE search 0
			 2017-09-10 02:41:46.905
			 No ptuse data
Processing 20170909-0031
PTUSE search 0
			 2017-09-10 03:02:52.370
			 No ptuse data
Processing 20170909-0033
PTUSE search 0
			 2017-09-10 03:20:43.917
			 No ptuse data
Processing 20170909-0034
PTUSE search 0
			 2017-09-10 03:41:45.883
			 No ptuse data
Processing 20170909-0035
PTUSE search 0
			 2017-09-10 04:02:52.347
			 No ptuse data
Processing 20170909-0037
PTUSE search 0
			 2017-09-10 04:20:46.393
			 No ptuse data
Processing 20170909-0038
PTUSE search 0
			 2017-09-10 04:41:50.358
			 No ptuse data
Processing 20170909-0039
PTUSE search 0
			 2017-09-10 05:02:52.824
			 No ptuse data
Processing 20170909-0041
PTUSE search 0
			 2017-09-10 05:20:44.370
			 No ptuse data
Processing 20170909-0042
PTUSE search 0
			 2017-09-10 05:41:49.835
			 No ptuse data
Processing 20170909-0043
PTUSE search 0
			 2017-09-10 06:02:54.800
			 No ptus

PTUSE search 1
			 J0437-4715.20180118T191650
Processing 20180118-0059
PTUSE search 1
			 J0437-4715.20180119T000700
Processing 20180119-0021
PTUSE search 1
			 J0437-4715.20180119T200140
Processing 20180119-0029
PTUSE search 1
			 J0437-4715.20180119T211220
Processing 20180120-0039
PTUSE search 1
			 J0437-4715.20180120T192450
Processing 20180127-0037
PTUSE search 1
			 J0437-4715.20180128T001056
Processing 20180201-0051
PTUSE search 1
			 J0437-4715.20180201T200008
Processing 20180201-0053


PTUSE search 1
			 J0437-4715.20180201T203144
Processing 20180206-0022
PTUSE search 1
			 J0437-4715.20180206T175104


In [ ]:
VisFile='/var/kat/archive2/data/MeerKATAR1/telescope_products/2017/08/28/1503947437.h5'
#VisFile=/var/kat/archive3/data/MeerKATAR1/telescope_products/2018/01/10/1515602236.h5
#    VisFile=''/var/kat/archive3/data/MeerKATAR1/telescope_products/2018/01/10/151560223
VisFile='/var/kat/archive3/data/MeerKATAR1/telescope_products/2018/01/10/1515602236.h5'
VisFile='/var/kat/archive3/data/MeerKATAR1/telescope_products/2018/01/16/1516133431.h5'

In [ ]:
data = katdal.open(VisFile)

In [ ]:
data = katdal.open(VisFile)
array_ant = katpoint.Antenna('ref', *(data.ants[0].ref_position_wgs84))
correlator_model = katpoint.DelayCorrection(data.ants, array_ant)
delay=correlator_model.max_delay
delay2=correlator_model.extra_delay
print array_ant
print delay2
print array_ant.name
refant=data.ants[0]
print refant.name
print refant.delay_model.params['FIX_V']
print refant.delay_model

In [ ]:
import h5py


In [ ]:
d=h5py.File('/var/kat/archive3/data/MeerKATAR1/telescope_products/2018/01/16/1516133431.h5')

In [ ]:
for ant in data.ants:
    print ant.name,ant.delay_model

In [ ]:
print data.ants

In [ ]:
print data.ants

In [ ]:
print data.ants

In [ ]:
print data.ants

In [ ]:
h = '<table>'
h += "<tr><td>Owner<td>UT Start<td>Schedule block<td>Source<td>Description<td>Antennas<td>NAnts<td>Duration<td>BW<td>CFreq<td>bfi1<td>size<td>ptuse<td>size<td>Archive</tr>"
#Owner	UT Start	Schedule block	Source	Description	Antennas	NAnts	Duration	BW	CFreq	bfi1 pulsar timing		ptuse pulsar timing		bfi1 pulsar search		visibility data		Phase-up				Instruction set					
for r in ar_res.docs:
    Observer = r['Observer']
    SB = r['ScheduleBlockIdCode']
    #print SB
    #print r['Targets']
    
    #link = 'http://kat-archive.kat.ac.za:8080/archive_search/#{"ei":"%s","so":true,"r":true,"k":true,"a":true,"b":true,"ob":"1","rs":{}}'%(SB)
    
    #link = 'http://kat-archive.kat.ac.za:8080/archive_search/#{"ei":"%s","so":true,"r":true,"k":true,"a":true,"b":true,"ob":"1","rs":{}}'%(SB)
    link = 'http://kat-archive.kat.ac.za:8080/archive_search/?{"ei":"%s","so":true,"a":true,"b":true,"ob":false,"rs":{}}'%(SB)

    #link = "http://kat-archive.kat.ac.za:8080/archive_search/#ei':'%s','so':true,'r':true,'k':true,'a':true,'b':true,'ob':'1','rs':{}}"%(SB)
    
    search_str = []
    search_str.append('ScheduleBlockIdCode:"%s"'%SB)

    prod_str=[]
    prod_str.append('CAS.ProductTypeName:PulsarTimingArchiveProduct')
    prod_str.append('CAS.ProductTypeName:PTUSETimingArchiveProduct')
    prod_str.append('CAS.ProductTypeName:PulsarSearchProduct')

    prod_string = '('+ ' OR '.join(prod_str) + ')'
    search_str.append(prod_string)
    #print prod_string
    search_string = ' AND '.join(search_str)
    #print search_string

    sb_res = archive.search( search_string ,rows=10000)
        #sb_res = archive.search(r['ScheduleBlockIdCode']  ,rows=10000,**{'sort':'StartTime asc'})
    #print r['Targets']
    ptpData = None
    spData = None
    ptuseData = None
    ptpFileSize = 0.0
    ptuseFileSize = 0.0
    spFileSize = 0.0
    for s in sb_res.docs:
        if s['CAS.ProductTypeName']=='PulsarTimingArchiveProduct':
            
            ptpData=s['CAS.ReferenceDatastore'][0][5:]
            ptpFileSize=sum(s['CAS.ReferenceFileSize'])/1024.0**3
        elif s['CAS.ProductTypeName']=='PTUSETimingArchiveProduct':
            
            ptuseData=s['CAS.ReferenceDatastore'][0][5:]
            ptuseFileSize=sum(s['CAS.ReferenceFileSize'])/1024.0**3
        elif s['CAS.ProductTypeName']=='PulsarSearchProduct':
        
            spData=s['CAS.ReferenceDatastore'][0][5:]
            spFileSize=sum(s['CAS.ReferenceFileSize'])/1024.0**3
    
        #print new2.loc[sb]['StartTime']
        
    phaseres = archive.search('CAS.ProductTypeName:MeerKATAR1TelescopeProduct AND InstructionSet:bf_phaseup* AND StartTime:[* TO %s]' % (r['StartTime']), sort="StartTime desc")
    phasesb = res2.docs[0]['ScheduleBlockIdCode']
    #print res2.docs[0]['StartTime'],res2.docs[0]['ScheduleBlockIdCode'],res2.docs[0]['ProductName']
    res3 = archive.search('CAS.ProductTypeName:MeerKATAR1ReductionProduct AND ExperimentID:%s' % (res2.docs[0]['ScheduleBlockIdCode']), sort="StartTime desc")
    obs_rep_link="http://kat-archive.kat.ac.za/archive2/data/MeerKATAR1/reduction_products/"+res3.docs[0]['ProductName']+'/obs_report_'+res2.docs[0]['ProductName']+".pdf"
    #print obs_rep_link
    #<a href=http://kat-archive.kat.ac.za:8080/archive_search/#{"s":"\\"%s\\"","e":"2/7/2017","a":true,"rs":{}}>%s</a>
    PhaseUpTime=res2.docs[0]['StartTime']
    PhaseUpSBid=res2.docs[0]['ScheduleBlockIdCode']
    #PhaseUplink = 'http://kat-archive.kat.ac.za:8080/archive_search/#{"ei":"%s","so":true,"r":true,"k":true,"a":true,"b":true,"ob":"1","rs":{}}'%(PhaseUpSBid)
    PhaseUplink = 'http://kat-archive.kat.ac.za:8080/archive_search/#{\"ei\":\"%s\",\"so\":true,\"r\":true,\"k\":true,\"a\":true,\"b\":true,\"ob\":\"1\",\"rs\":{}}'%(PhaseUpSBid)
    if (Observer != 'CAM') and (Observer != 'ruby'):
        try:
            #h += '<tr> <td> %s <td> %s <td> <a href=\'%s\'>%s</a> <td> %s <td> %s <td> %s <td> %d'%(r['Observer'],r['StartTime'],link,SB,r['Targets'][0],r['Description'], r['Antennas'],len(r['Antennas']))
            h += '<tr> <td> %s <td> %s <td> %s <td> %s <td> %s <td> %s <td> %s <td> %d'%(r['Observer'],r['StartTime'],SB,r['Targets'][0],link,r['Description'], r['Antennas'],len(r['Antennas']))
           
            h += '<td> %5.2f <td> %d <td> %5.2f' %(r['Duration'],(r['NumFreqChannels']*r['ChannelWidth'])/1.0E6,r['CenterFrequency']*1E-6)
            h += '<td> %4.2f'%(r['DumpPeriod'])
            h += '<td> %s <td> %5.2f <td> %s <td> %5.2f <td> %s <td> %5.2f'%(ptpData,ptpFileSize,ptuseData,ptuseFileSize,spData,spFileSize)
            h += '<td> %s <td> %5.2f' %(r['CAS.ReferenceDatastore'][0][5:],r['FileSize'][0]/1024.0**3)                                                                                                     
            #h += '<td> <a href=\'%s\'>%s</a>'%(obs_rep_link,'Obs-rep_'+PhaseUpSBid)
            h += '<td> =hyperlink("%s","%s")'%(obs_rep_link,'Obs-rep_'+PhaseUpSBid)
            h += '<td> %s'%(PhaseUpTime)
            h += '<td> %s'%(PhaseUplink)
            #h += '<td> =hyperlink("%s","%s")'%(PhaseUpLink,PhaseUpSBid)
            h += '<td> %s <td> %s <td> %s'%(" "," "," ")
            h += '<td> %s'%(r['InstructionSet'])
            h += '<td> %s'%(r['CAS.ProductId'])
            h += '</tr>'
            
        except:
            continue

h += "</table>"
f.write(h)
f.close()